In [ ]:
!pip install -U ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO
print("YOLOv8 ready")

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")


In [ ]:
# Train the model
results = model.train(data="coco8.yaml", batch=4, epochs=50, imgsz=640)

In [ ]:
!zip -r train.zip /content/runs/detect/train6

In [ ]:
from google.colab import files
files.download("/content/train.zip")


# Inference

In [ ]:
import os
import cv2
from ultralytics import YOLO

# -----------------------------
# CONFIG
# -----------------------------
MODEL_PATH = "/content/runs/detect/train6/weights/best.pt"
IMAGE_DIR = "/content/drive/MyDrive/ML-Project/ui-element-detection/images"
OUTPUT_DIR = "/content/sample_data/output"
CONF_THRESHOLD = 0.1

CLASS_NAMES = {
    0: "table_column",
    1: "text_field",
    2: "text_info"
}

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------------
# LOAD MODEL
# -----------------------------
model = YOLO(MODEL_PATH)

# -----------------------------
# RUN INFERENCE
# -----------------------------
results = model(
    source=IMAGE_DIR,
    conf=CONF_THRESHOLD,
    save=False,
    stream=True  # memory efficient
)

# -----------------------------
# DRAW & SAVE
# -----------------------------
for result in results:
    img = result.orig_img
    h, w, _ = img.shape

    if result.boxes is not None:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])

            label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"

            # Draw box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw label
            cv2.putText(
                img,
                label,
                (x1, max(y1 - 10, 20)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2
            )

    # Save output
    output_path = os.path.join(OUTPUT_DIR, os.path.basename(result.path))
    cv2.imwrite(output_path, img)

print("Inference completed. Annotated images saved to:", OUTPUT_DIR)
